# C. PERFORMANCE UNDER NEW CATEGORIES
In this experiment, we will evaluate effect of the proposed few-shot learning method to address the fourth challenge in limited data fault diagnosis: 
especially in real-world applications, fault categories and working conditions  are usually unbalanced. It is thus difficult to collect enough samples for every fault type under different working conditions. We mainly pay attention to unbalanced fault categories. When new categories appear, traditional deep learning methods need to retrain to deal with the new categories diagnosis. However, different from the traditional deep learning methods,  the few-shot learning model can be directly used in the new  category diagnosis by just giving a few new category samples. We train the WDCNN model from all categories and the few-shot model from  30\% randomly new categories to all see categories. We repeated each such experiment ten times to deal with the randomness of the algorithms.  



In [1]:
%matplotlib inline
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
# set the memory usage
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
set_session(tf.Session(config=tf_config))

import matplotlib.pyplot as plt
from scipy.io import loadmat
import numpy as np
import imp
import pandas as pd


Using TensorFlow backend.


## load data

In [2]:
import cwru 

window_size = 2048
data = cwru.CWRU(['12DriveEndFault'], ['1772', '1750', '1730'], window_size)
data.nclasses,data.classes,len(data.X_train),len(data.X_test)

Datasets/CWRU 12DriveEndFault 1730
0 Datasets/CWRU/NormalBaseline/1730/Normal.mat
(485643, 2)
1 Datasets/CWRU/NormalBaseline/1750/Normal.mat
dict_keys(['__globals__', '__version__', 'X098_DE_time', 'X099_DE_time', '__header__', 'X099_FE_time', 'ans', 'X098_FE_time'])
(485063, 2)
2 Datasets/CWRU/NormalBaseline/1772/Normal.mat
(483903, 2)
3 Datasets/CWRU/12DriveEndFault/1730/0.007-Ball.mat
(121556, 2)
4 Datasets/CWRU/12DriveEndFault/1750/0.007-Ball.mat
(121556, 2)
5 Datasets/CWRU/12DriveEndFault/1772/0.007-Ball.mat
(121410, 2)
6 Datasets/CWRU/12DriveEndFault/1730/0.014-Ball.mat
(122136, 2)
7 Datasets/CWRU/12DriveEndFault/1750/0.014-Ball.mat
(121991, 2)
8 Datasets/CWRU/12DriveEndFault/1772/0.014-Ball.mat
(122136, 2)
9 Datasets/CWRU/12DriveEndFault/1730/0.021-Ball.mat
(122136, 2)
10 Datasets/CWRU/12DriveEndFault/1750/0.021-Ball.mat
(122136, 2)
11 Datasets/CWRU/12DriveEndFault/1772/0.021-Ball.mat
(121701, 2)
12 Datasets/CWRU/12DriveEndFault/1730/0.007-InnerRace.mat
(122917, 2)
13 Datasets/C

(10,
 [('NormalBaselineNormal', 0),
  ('12DriveEndFault0.007-Ball', 1),
  ('12DriveEndFault0.014-Ball', 2),
  ('12DriveEndFault0.021-Ball', 3),
  ('12DriveEndFault0.007-InnerRace', 4),
  ('12DriveEndFault0.014-InnerRace', 5),
  ('12DriveEndFault0.021-InnerRace', 6),
  ('12DriveEndFault0.007-OuterRace6', 7),
  ('12DriveEndFault0.014-OuterRace6', 8),
  ('12DriveEndFault0.021-OuterRace6', 9)],
 19800,
 750)

## Load models

In [3]:
import models
# imp.reload(models)
siamese_net = models.load_siamese_net((window_size,2))
print('\nsiamese_net summary:')
siamese_net.summary()


siamese_net summary:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2048, 2)      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2048, 2)      0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 100)          51796       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 100)          0           sequential_1[1][0]    

In [4]:
print('\nsequential_3 is WDCNN:')
siamese_net.layers[2].summary()


sequential_3 is WDCNN:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 128, 16)           2064      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 16)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 64, 32)            1568      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 32, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 32, 64)            4160      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 16, 64)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 16, 64)         

In [5]:
wdcnn_net = models.load_wdcnn_net()
print('\nwdcnn_net summary:')
wdcnn_net.summary()

52806

wdcnn_net summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 2048, 2)           0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 100)               51796     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1010      
Total params: 52,806
Trainable params: 52,806
Non-trainable params: 0
_________________________________________________________________


## Config

In [6]:
import keras
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping

import siamese
# imp.reload(siamese)
import utils
imp.reload(utils)

settings = {
  "N_way": 10,           # how many classes for testing one-shot tasks>
  "batch_size": 32,
  "best": -1,
  "evaluate_every": 600,   # interval for evaluating on one-shot tasks
  "loss_every": 20,      # interval for printing loss (iterations)
  "n_iter": 15000,
  "n_val": 2,          #how many one-shot tasks to validate on?
  "n": 0,
  "save_path":"",
  "save_weights_file": "weights-best-10-oneshot-low-data.hdf5"
}


exp_name = "EXP-C"
exps = [90,19800]
see_rates = [0.7,0.8,0.9]
times = 10

## Training & Testing

In [ ]:
is_training = False    # enable or disable train models. if enable training, save best models will be update.

train_classes = sorted(list(set(data.y_train)))
train_indices = [np.where(data.y_train == i)[0] for i in train_classes]
for exp in exps:
    # enable the random seed for experimental reproduction
    np.random.seed(exp)
    scores_1_shot = []
    scores_5_shot = []
    num = int(exp/len(train_classes))
    settings['evaluate_every'] = 300 if exp<1000 else 600
    print(settings['evaluate_every'])
    for time_idx in range(times):
        np.random.seed(0)
        train_idxs = []
        val_idxs = []
        for i, c in enumerate(train_classes):
            select_idx = train_indices[i][np.random.choice(len(train_indices[i]), num, replace=False)]
            split = int(0.6*num)
            train_idxs.extend(select_idx[:split])
            end = num
            if(0.4*num>100):
                end = split+100
            val_idxs.extend(select_idx[split:end])

        X_train, y_train = data.X_train[train_idxs],data.y_train[train_idxs] 
        X_val, y_val = data.X_train[val_idxs],data.y_train[val_idxs] 

        print(train_idxs[0:10])
        print(val_idxs[0:10])   
            
        random_train_classes = [0]+list(np.random.choice(list(range(1,10)), 9, replace=False))
        for rate in see_rates:
            print("\n%s-%s-%s"%(exp,rate,time_idx) + '*'*80)
            settings["save_path"] = "tmp/%s/size_%s/%s/time_%s/" % (exp_name,exp,rate,time_idx)
            data._mkdir(settings["save_path"])
            random_select_classes = random_train_classes[:int(rate*len(train_classes))]
            print(random_select_classes)
            train_idxs_rate = np.array([np.where(y_train == i)[0] for i in random_select_classes]).reshape(-1)
            val_idxs_rate =  np.array([np.where(y_val == i)[0] for i in random_select_classes]).reshape(-1)
            X_train_rate, y_train_rate = X_train[train_idxs_rate],y_train[train_idxs_rate]
            X_val_rate, y_val_rate = X_val[val_idxs_rate],y_val[val_idxs_rate]

            # load one-shot model and training
            siamese_net = models.load_siamese_net()
            siamese_loader = siamese.Siamese_Loader(X_train_rate,
                                            y_train_rate,
                                            X_val_rate,
                                            y_val_rate)

            if(is_training):
                print(siamese.train_and_test_oneshot(settings,siamese_net,siamese_loader))
                
            print("load best weights",settings["save_path"] + settings['save_weights_file'])
            siamese_net.load_weights(settings["save_path"] + settings['save_weights_file'])


            siamese_loader = siamese.Siamese_Loader(X_train,
                                            y_train,
                                            data.X_test,
                                            data.y_test)
            s = 'val'
            preds_5_shot = []
            scores = []
            for k in range(5):
                val_acc,preds = siamese_loader.test_oneshot2(siamese_net,len(siamese_loader.classes[s]),
                                                             len(siamese_loader.data[s]),verbose=False)
    #             confusion_plot(preds[:,1],preds[:,0])
                print(val_acc)
                scores.append(val_acc)
                preds_5_shot.append(preds[:,1])
            preds = []
            for line in np.array(preds_5_shot).T:
                preds.append(np.argmax(np.bincount(line)))
    #         confusion_plot(np.array(preds),data.y_test) 
            score_5_shot = accuracy_score(data.y_test,np.array(preds))*100
            print('5_shot:',score_5_shot)
            scores_1_shot.append(scores[0])
            scores_5_shot.append(score_5_shot)

    a =pd.DataFrame(np.array(scores_1_shot).reshape(-1,len(see_rates)))
    a.columns = see_rates
    a.to_csv("tmp/%s/size_%s/scores_1_shot.csv" % (exp_name,exp),index=True)


    a =pd.DataFrame(np.array(scores_5_shot).reshape(-1,len(see_rates)))
    a.columns = see_rates
    a.to_csv("tmp/%s/size_%s/scores_5_shot.csv" % (exp_name,exp),index=True)

## Analysis

In [7]:
exps = [90,19800]

scores_1_shot_all = pd.DataFrame()
scores_5_shot_all = pd.DataFrame()

for exp in exps:
    file_path = "tmp/%s/size_%s" % (exp_name,exp)
    tmp_data = pd.read_csv("%s/scores_1_shot.csv" % (file_path), 
                           sep=',', index_col=0)
    tmp_data = pd.DataFrame(tmp_data.values.reshape(-1,len(see_rates)))
    tmp_data.columns = see_rates
    tmp_data['model'] = 'One-shot'
    tmp_data['exp'] = exp 
    scores_1_shot_all = pd.concat([scores_1_shot_all,tmp_data],axis=0)

    tmp_data = pd.read_csv("%s/scores_5_shot.csv" % (file_path), 
                           sep=',', index_col=0)
    tmp_data = pd.DataFrame(tmp_data.values.reshape(-1,len(see_rates)))
    tmp_data.columns = see_rates
    tmp_data['model'] = 'Five-shot'
    tmp_data['exp'] = exp 
    scores_5_shot_all = pd.concat([scores_5_shot_all,tmp_data],axis=0)


scores_1_shot_all.to_csv("tmp/%s/scores_1_shot_all.csv" % (exp_name), float_format='%.6f', index=True)
scores_5_shot_all.to_csv("tmp/%s/scores_5_shot_all.csv" % (exp_name), float_format='%.6f', index=True)

scores_1_shot_all['model'] = 'One-shot'
scores_5_shot_all['model'] = 'Five-shot'

scores_all = pd.concat([scores_1_shot_all,scores_5_shot_all],axis=0)
scores_all.to_csv("tmp/%s/scores_all.csv" % (exp_name), float_format='%.6f', index=True)   

scores_all

,0.7,0.8,0.9,model,exp
0,65.733333,80.266667,86.400000,One-shot,90
1,72.800000,82.800000,86.133333,One-shot,90
2,62.933333,79.600000,90.400000,One-shot,90
3,62.400000,79.466667,89.333333,One-shot,90
4,72.666667,79.200000,82.400000,One-shot,90
5,59.866667,83.066667,92.133333,One-shot,90
6,60.533333,80.533333,85.333333,One-shot,90
7,61.200000,81.866667,92.400000,One-shot,90
8,58.666667,87.466667,82.133333,One-shot,90
9,61.066667,78.000000,83.733333,One-shot,90


In [8]:
scores_all_mean = scores_all.groupby(['exp','model']).mean()\
                .sort_values(by=['model'],ascending=0).sort_values(by=['exp'],ascending=1)
scores_all_std = scores_all.groupby(['exp','model']).std()\
                .sort_values(by=['model'],ascending=0).sort_values(by=['exp'],ascending=1)
scores_all_mean.to_csv("tmp/%s/scores_all_mean.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_std.to_csv("tmp/%s/scores_all_std.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_mean, scores_all_std

(                       0.7        0.8        0.9
 exp   model                                     
 90    One-shot   63.786667  81.226667  87.040000
       Five-shot  64.813333  83.040000  88.200000
 19800 One-shot   82.226667  91.693333  96.013333
       Five-shot  83.613333  92.973333  96.106667,
                       0.7       0.8       0.9
 exp   model                                  
 90    One-shot   5.085812  2.725862  3.828316
       Five-shot  6.381594  2.505342  3.475914
 19800 One-shot   2.769891  3.267830  2.459710
       Five-shot  3.639265  3.028131  2.849163)